Import Libraries

In [ ]:
import os
import re
import sys
import nltk
from google.colab import drive
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Embedding
from keras.layers import LSTM
try:
    from nltk.corpus import stopwords
except:
    nltk.download('stopwords')
    from nltk.corpus import stopwords

Method for Reading Data

In [ ]:
def read_data(path, x_data, y_data):

  for filename in os.listdir(path):
        with open(os.path.join(path, filename), 'r', encoding="utf8") as f:
            st_ind = list(filename).index("_")
            nd_ind = list(filename).index(".")
            if filename[nd_ind - 1] == ')':
                nd_ind -= 4 
            x_data.append(f.read())
            y_data.append(int(filename[st_ind + 1:nd_ind]) / 10)

  return x_data, y_data

Method for importing dataset

In [ ]:
def import_data():

    print("Importing Dataset... Please wait!")

    dataset_path = "/content/gdrive/My Drive/Colab Notebooks/DataSet/"

    test_data_path = dataset_path + "Test/"
    pos_test_data_path = test_data_path + "Positive"
    neg_test_data_path = test_data_path + "Negative"

    train_data_path = dataset_path + "Train/"
    pos_train_data_path = train_data_path + "Positive"
    neg_train_data_path = train_data_path + "Negative"

    x_train, y_train = read_data(neg_train_data_path, [], [])
    x_train, y_train = read_data(pos_train_data_path, x_train, y_train)

    x_test, y_test = read_data(neg_test_data_path, [], [])
    x_test, y_test = read_data(pos_test_data_path, x_test, y_test)


    print("Dataset successfully imported...!!!")
    return x_train, y_train, x_test, y_test

Method for Data Pre-processing

In [ ]:
def preProcessData(x_data):
        
        print("Pre-processing data... Please wait!")
        
        #Fetch all stopwords and keep required stopwords
        all_stopwords = stopwords.words('english')
        my_stopwords = [ word for word in all_stopwords if word not in ("against", "up", "down", "out", "off", "over", "under", "more", "most", "each", "few", "some", "such", "no", "nor", "not", "only", "too", "very", "don", "don't", 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't")]
        
        #Get rid of special characters
        REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
        REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
        
        for i in range(0,len(x_data)):
            #Keep only alphabets with single whitespace
            x_data[i] = REPLACE_NO_SPACE.sub("", x_data[i].lower())
            x_data[i] = REPLACE_WITH_SPACE.sub(" ", x_data[i])
            
            #Remove unwanted stopwords
            x_data[i] = x_data[i].split()
            x_data[i] = [ word for word in x_data[i] if word not in my_stopwords]
            x_data[i] = " ".join(x_data[i])
 
        print("Pre-processing done...!!!")
        return x_data

Method for Word Embedding - One-Hot Encoding

In [ ]:
def encodeData(max_features, max_doc_len, x_data):
        
        print("Encoding data to One Hot Representation...")
        
        x_data = [ one_hot(document, max_features) for document in x_data]
        
        #Add Bias
        for i in range(0, len(x_data)):
            x_data[i] = [1] + x_data[i]
        
        #Word Embedding
        x_data = pad_sequences(x_data, truncating = 'post', padding = 'post', maxlen = max_doc_len)
        
        print("One-Hot Representation Encoding done...!!!")
        return x_data

Method for creating ML->RNN->LSTM model

In [ ]:
def createModel(max_features):
        
        print("Creating model...")

        model = Sequential()
        #Layer 1-> Embedding
        model.add(Embedding(max_features, 128))     // 32 * 200 * 128
        #Layer 2-> LSTM
        model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))    // 32 * 128 
        #Layer 3-> Fully Connected (Dense)
        model.add(Dense(1, activation='sigmoid'))
        #Choose best optimizer
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        print("Model created...!!!")
        return model

Method for training the model

In [ ]:
def trainModel(model, x_train, y_train, x_test, y_test):
        
        print("Training Model... Please wait!")

        model.fit(x_train, y_train, batch_size=32, epochs=15, validation_data=(x_test, y_test))
        
        print("Model trained...!!!")
        return model

Method for evaluating the model

In [ ]:
def validateModel(model, x_test, y_test):

        score, acc = model.evaluate(x_test, y_test, batch_size=32)
        print('Test score:', score)
        print('Test accuracy:', acc)

Method for Prediction

In [ ]:
def predictModel(model, x_data):

  print(model.predict(x_data))

Method for saving trained model

In [ ]:
def saveModel(model):

        #Serialize to JSON
        json_file = model.to_json()
        with open("Model_LSTM.json", "w") as file:
            file.write(json_file)
        
        #Serialize weights to HDF5
        model.save_weights("lstm_model_weights.h5")
        
        print("Model saved...")

Method for loading saved model

In [ ]:
def loadModel():

        #Load JSON and create model
        file = open("Model_LSTM.json", "r")
        model_json = file.read()
        file.close()
        
        model = model_from_json(model_json)
        #Load weights
        model.load_weights("lstm_model_weights.h5")
        print("Model loaded successfully...")
        
        return model

Main Method

In [ ]:
def main():

  #Define vocabulary size
  max_features = 47000
  #Define number of words per document/review
  max_doc_len = 220

  x_train, y_train, x_test, y_test = import_data()

  x_train = preProcessData(x_train)
  x_test = preProcessData(x_test)

  x_train = encodeData(max_features, max_doc_len, x_train)
  x_test = encodeData(max_features, max_doc_len, x_test)

  model = createModel(max_features)

  model = trainModel(model, x_train, y_train, x_test, y_test)

  validateModel(model, x_test, y_test)

  saveModel(model)

  model = loadModel()

  # Predict new document
  new_document = ["""I absolutely adored this movie. For me, the best reason to see it is how stark a contrast it is from legal dramas like "Boston Legal" or "Ally McBeal" or even "LA Law." This is REALITY. The law is not BS, won in some closing argument or through some ridiculous defense you pull out of your butt, like the "Chewbacca defense" on South Park.) This is a real travesty of justice, the legal system gone horribly wrong, and the work by GOOD lawyers - not the shyster stereotype, who use all of their skills to right it. It will do more for restoring your faith in humanity than any Frank Capra movie or TO KILL A MOCKINGBIRD. And most importantly, I wept. During the film, during the featurette included at the end of the DVD - it's amazing. Wonderful film; wonderfully made. Thank God the filmmakers made it."""]

  new_document = preProcessData(new_document)

  new_document = encodeData(max_features, max_doc_len, new_document)

  predictModel(model, new_document)


Call Main Method

In [ ]:
if __name__ == "__main__":
    main()

Importing Dataset... Please wait!


OSError: ignored